In [1]:
import folium
import pandas
import json
from folium import GeoJsonTooltip
import IPython
from IPython.display import IFrame
import bs4
import base64

In [2]:
MAP_FILE = "map_co2.html"

# Information on power plants
power_plant_data = pandas.read_csv("global_power_plant_database.csv")
power_plant_type = list(power_plant_data["primary_fuel"])
power_plant_name = list(power_plant_data["name"])
power_plant_latitude = list(power_plant_data["latitude"])
power_plant_longitude = list(power_plant_data["longitude"])
power_plant_capacity = list(power_plant_data["capacity_mw"])

html_power_plant = f"""
Power Plant Name: <br>
<a href="https://www.google.com/search?q=%%22%s%%22" target="_blank">%s</a><br>
Primary Fuel: %s <br>
Capacity: %s MW
"""


# Function to find average capacity of power plants
def find_avg_capacity(energy_type):
    total_capacity = []
    for power, capacity in zip(power_plant_type, power_plant_capacity):
        if power == energy_type:
            total_capacity.append(capacity)
    return (sum(total_capacity)) / len(total_capacity)


# Function to add power plants
def add_power_plants(feature_group, energy_type, color):
    for power, name, lat, lon, capacity in zip(power_plant_type, power_plant_name, power_plant_latitude,
                                               power_plant_longitude, power_plant_capacity):
        if power == energy_type and capacity >= find_avg_capacity(energy_type=power):
            iframe = folium.IFrame(html=html_power_plant % (name, name, power, capacity), width=200, height=100)
            feature_group.add_child(folium.CircleMarker(location=(lat, lon), radius=0.2, fill=True,
                                                        fill_color=color, fill_opacity=1, color=color,
                                                        popup=folium.Popup(iframe)
                                                        ))


# Base map
emissions_map = folium.Map(zoom_start=2, min_zoom=2, tiles="cartodbdark_matter", max_bounds=True,
                           width='100%', height='100%')

# CO2 emissions feature group
fg_emissions = folium.FeatureGroup(name="CO2 Emissions 2019")
co2_data = open('total_world_co2_2019.json', 'r', encoding='utf-8-sig')
fg_emissions.add_child(folium.GeoJson(co2_data.read(),
                                      style_function=lambda x: {'weight': 1, 'color': 'gray', 'fillColor': '#fccbb8'
                                      if x['properties']['EMISSIONS'] < 100
                                      else '#ff8059' if x['properties']['EMISSIONS'] < 500
                                      else '#ff7a7a' if x['properties']['EMISSIONS'] < 1000
                                      else '#ff4545' if x['properties']['EMISSIONS'] < 5000
                                      else '#ff0303' if x['properties']['EMISSIONS'] < 7500
                                      else '#850000'},
                                      highlight_function=lambda x: {'weight': 2, 'color': '#adadad'},
                                      tooltip=folium.features.GeoJsonTooltip(fields=['NAME', 'EMISSIONS'],
                                                                             aliases=['Country', '2019 CO2 Emissions(t)'],
                                                                             labels=True,
                                                                             sticky=True,
                                                                             toLocaleString=True)))


# Feature groups for power plants
fg_coal_plants = folium.FeatureGroup(name="Coal Power Plants")
fg_natural_gas = folium.FeatureGroup(name="Natural Gas Power Plants")
fg_nuclear_plants = folium.FeatureGroup(name="Nuclear Power Plants")
fg_biomass_plants = folium.FeatureGroup(name="Biofuels/Biomass Plants")
fg_oil_plants = folium.FeatureGroup(name="Oil Plants")

add_power_plants(fg_coal_plants, "Coal", "#ff6600")
add_power_plants(fg_natural_gas, "Gas", "#c70067")
add_power_plants(fg_nuclear_plants, "Nuclear", "#ff0000")
add_power_plants(fg_biomass_plants, "Biomass", "#4dff00")
add_power_plants(fg_oil_plants, "Oil", "#0095a6")

# Adding to base map
emissions_map.add_child(fg_emissions)
emissions_map.add_child(fg_coal_plants)
emissions_map.add_child(fg_natural_gas)
emissions_map.add_child(fg_nuclear_plants)
emissions_map.add_child(fg_biomass_plants)
emissions_map.add_child(fg_oil_plants)
emissions_map.add_child(folium.LayerControl())
emissions_map.save(MAP_FILE)

In [3]:
# Displaying initial map
IFrame(src=MAP_FILE, width=900, height=600)

In [4]:
# Setting map bounds
with open(MAP_FILE, mode="r+") as h:
    text1 = h.read()

with open(MAP_FILE, mode="w") as h:
    h.write(text1.replace('"noWrap": false', '"noWrap": true'))


# Creating soup object
with open(MAP_FILE, mode="r+") as h:
    text2 = h.read()

soup = bs4.BeautifulSoup(text2, "lxml")


# Adding top heading for web page
heading_tag = soup.new_tag('h1')
heading_tag['class'] = "map-heading-style"
heading_tag.string = 'CO2 Emissions 2019'
soup.body.insert(1, heading_tag)


# Style tag for heading
head = soup.head
st = soup.new_tag('style', type='text/css')
st.append('''
.folium-map {
	background: #262626 !important;
}

.map-heading-style {
    margin: 0;
    background: #262626;
    color: white;
	text-align: center;
    padding: 15px;
}

.legend-image {
    display: block;
    margin: auto !important;
}
''')
head.insert(5, st)


# Style tag for web page background color
bg = soup.new_tag('style')
bg.append('html, body {background:#262626 !important;}')
head.insert(6, bg)


# Adding legend picture

legend_tag = soup.new_tag('img')
legend_tag['class'] = "map-heading-style legend-image"

with open("co2_map_legend.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")  
legend_tag['src'] = "data:image/png;base64," + encoded_string

legend_tag['alt'] = "CO2 Map Legend"
soup.body.insert(2, legend_tag)


modified_html = soup.prettify("utf-8")


# Writing final file
with open("index.html", "wb") as file:
    file.write(modified_html)

In [5]:
# Displaying final map
IFrame(src="index.html", width=900, height=600)